In [ ]:
import kagglehub

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("juliusshrestha/wilms-tumorct")

print("Path to dataset files:", path)

100%|██████████| 776M/776M [00:07<00:00, 111MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/versions/1


In [ ]:
!pip install torch torchvision matplotlib pillow scikit-learn


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image directory
image_dir = "/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/versions/1"

# Create ImageDataGenerator for normalization and augmentation
datagen = ImageDataGenerator(
    rescale=1./255,                 # Normalize to [0, 1]
    rotation_range=30,              # Random rotations
    width_shift_range=0.2,          # Random horizontal shifts
    height_shift_range=0.2,         # Random vertical shifts
    shear_range=0.2,                # Random shearing
    zoom_range=0.2,                 # Random zooms
    horizontal_flip=True,           # Random horizontal flip
    fill_mode='nearest'             # Pixel filling method for rotated areas
)

# Load images using flow_from_directory (make sure the dataset is organized into subfolders for each class)
train_generator = datagen.flow_from_directory(
    image_dir,
    target_size=(224, 224),          # Resize images to 224x224 pixels
    batch_size=32,
    class_mode='binary',             # Binary labels, can be 'categorical' for multi-class
    shuffle=True
)

# Example of accessing an image batch
images, labels = next(train_generator)
print(images.shape, labels.shape)


Found 23450 images belonging to 1 classes.
(32, 224, 224, 3) (32,)


In [ ]:
# Split dataset into training and validation sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% of the data will be used for validation
)

# Define the training set
train_generator = train_datagen.flow_from_directory(
    image_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # or 'categorical' if more than 1 class
    shuffle=True,
    subset='training'    # Indicating the training split
)

# Define the validation set
validation_generator = train_datagen.flow_from_directory(
    image_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    subset='validation'  # Indicating the validation split
)


Found 18760 images belonging to 1 classes.
Found 4690 images belonging to 1 classes.


In [ ]:
import cv2
import numpy as np

# Define a custom preprocessing function for histogram equalization
def equalize_histogram(image):
    # Convert image to grayscale for equalization (if you prefer to keep it in RGB, we can skip this step)
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Apply histogram equalization
    equalized_image = cv2.equalizeHist(gray_image)
    # Convert back to RGB by repeating the equalized grayscale image in 3 channels
    equalized_image_rgb = cv2.cvtColor(equalized_image, cv2.COLOR_GRAY2RGB)
    return equalized_image_rgb

# Create ImageDataGenerator with custom preprocessing function
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize to [0, 1]
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=equalize_histogram  # Apply the custom histogram equalization function
)

# Use the updated generator for loading and augmenting images
train_generator = datagen.flow_from_directory(
    image_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # For binary classification
    shuffle=True
)


Found 23450 images belonging to 1 classes.


In [ ]:
# Define a custom preprocessing function for denoising using Gaussian blur
def denoise_image(image):
    # Apply Gaussian blur to reduce noise
    denoised_image = cv2.GaussianBlur(image, (5, 5), 0)
    return denoised_image

# Create ImageDataGenerator with custom preprocessing function for denoising
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize to [0, 1]
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=denoise_image  # Apply the denoising function
)

# Use the updated generator for loading and augmenting images
train_generator = datagen.flow_from_directory(
    image_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # For binary classification
    shuffle=True
)


Found 23450 images belonging to 1 classes.


In [ ]:
# Import necessary modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization


In [ ]:
from tensorflow.keras.layers import BatchNormalization

# Build the updated CNN model
model = Sequential()

# First Convolutional Block
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Second Convolutional Block
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Third Convolutional Block
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Flatten the output from convolutional layers
model.add(Flatten())

# Fully connected layers
model.add(Dense(256, activation='relu'))  # Reduced the number of units
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summarize the model
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 111, 111, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 54, 54, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 26, 26, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      22,151,424 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,245,825 (84.86 MB)

 Trainable params: 22,245,377 (84.86 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
import kagglehub

# Download the latest version of the dataset again
path = kagglehub.dataset_download("juliusshrestha/wilms-tumorct")
print("Path to dataset files:", path)


Path to dataset files: /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/versions/1


In [ ]:
!pip install --upgrade kagglehub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: kagglehub
    Found existing installation: kagglehub 0.3.5
    Uninstalling kagglehub-0.3.5:
      Successfully uninstalled kagglehub-0.3.5


In [ ]:
# Define image directory after successful download
image_dir = "/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/versions/1"


In [ ]:
import os

# Check if the image directory exists
image_dir = "/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/versions/1"
if os.path.exists(image_dir):
    print("Directory exists!")
else:
    print("Directory does not exist!")


Directory exists!


In [ ]:
image_dir = "/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/versions/1"


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Define the data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% of the data will be used for validation
)

# Define the training set
train_generator = train_datagen.flow_from_directory(
    image_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # or 'categorical' if more than 1 class
    shuffle=True,
    subset='training'    # Indicating the training split
)

# Define the validation set
validation_generator = train_datagen.flow_from_directory(
    image_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    subset='validation'  # Indicating the validation split
)


Found 18760 images belonging to 1 classes.
Found 4690 images belonging to 1 classes.


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # You can adjust the number of epochs as per your needs
    validation_data=validation_generator
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 20/587 ━━━━━━━━━━━━━━━━━━━━ 40:53 4s/step - accuracy: 0.8938 - loss: 0.7228

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have true labels (y_true) and predictions (y_pred)
y_true = validation_generator.classes
y_pred = model.predict(validation_generator)
y_pred = (y_pred > 0.5).astype("int32")  # Convert probabilities to binary labels

cm = confusion_matrix(y_true, y_pred)

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No Tumor', 'Tumor'], yticklabels=['No Tumor', 'Tumor'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have true labels (y_true) and predictions (y_pred)
y_true = validation_generator.classes
y_pred = (model.predict(validation_generator) > 0.5).astype("int32")  # Convert probabilities to binary labels

# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix using seaborn heatmap
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No Tumor', 'Tumor'], yticklabels=['No Tumor', 'Tumor'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Print the metrics
print(f"Precision: {precision:.4f}")
print(f"Recall (Sensitivity): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


ResNet-50 model


In [ ]:
!pip install torch torchvision tqdm


In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm

class NKUTModel(nn.Module):
    def __init__(self, num_classes):
        super(NKUTModel, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}


In [ ]:
import os

base_path = "/root/.cache/kagglehub/datasets"

for root, dirs, files in os.walk(base_path):
    print(f"Root: {root}")
    print(f"Directories: {dirs}")
    print(f"Files: {files}")
    print("-" * 50)


Root: /root/.cache/kagglehub/datasets
Directories: ['juliusshrestha']
Files: []
--------------------------------------------------
Root: /root/.cache/kagglehub/datasets/juliusshrestha
Directories: ['wilms-tumorct']
Files: []
--------------------------------------------------
Root: /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct
Directories: ['versions']
Files: ['1.complete']
--------------------------------------------------
Root: /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/versions
Directories: ['1']
Files: []
--------------------------------------------------
Root: /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/versions/1
Directories: ['datasets']
Files: []
--------------------------------------------------
Root: /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/versions/1/datasets
Directories: ['test_images', 'val_images', 'test_masks', 'train_images', 'train_masks', 'val_masks']
Files: ['_classes.csv']
-------------------------

In [ ]:
!kaggle datasets download -d juliusshrestha/wilms-tumorct
!unzip wilms-tumorct.zip -d /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/


Streaming output truncated to the last 5000 lines.
  inflating: /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets/train_masks/cases3012_348.png  
  inflating: /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets/train_masks/cases3012_349.png  
  inflating: /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets/train_masks/cases3012_350.png  
  inflating: /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets/train_masks/cases3012_351.png  
  inflating: /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets/train_masks/cases3012_353.png  
  inflating: /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets/train_masks/cases3012_356.png  
  inflating: /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets/train_masks/cases3012_357.png  
  inflating: /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets/train_masks/cases3012_358.png  
  inflating: /root/.cache/kag

In [ ]:
import os

# Define the directory path
dataset_dir = "/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct"

# Create the directory if it doesn't exist
os.makedirs(dataset_dir, exist_ok=True)
print(f"Directory created at: {dataset_dir}")


Directory created at: /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct


In [ ]:
!unzip wilms-tumorct.zip -d /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/


Archive:  wilms-tumorct.zip
replace /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets/_classes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os

dataset_path = "/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct"
print("Contents of the extracted dataset:")
print(os.listdir(dataset_path))


Contents of the extracted dataset:
['versions', '1.complete', 'datasets']


In [ ]:
import os

extracted_path = "/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets"
print("Contents of the datasets folder:")
print(os.listdir(extracted_path))


Contents of the datasets folder:
['test_images', 'val_images', 'test_masks', 'train_images', 'train_masks', '_classes.csv', 'val_masks']


In [ ]:
dataset_path = "/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets"


In [ ]:
import os
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

# Define the Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_names = os.listdir(image_dir)  # List all image filenames in the directory

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name)  # Masks should have the same name as the images

        # Open the image and mask
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # Convert mask to grayscale

        if self.transform:
            image = self.transform(image)
            # Explicitly resize the mask
            mask = transforms.Resize((224, 224))(mask)
            mask = transforms.ToTensor()(mask)  # Convert mask to tensor without normalizing

        return image, mask

# Define transformations for resizing and converting images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images and masks to 224x224
    transforms.ToTensor(),  # Convert images to tensors
])

# Define dataset paths based on your directory structure
dataset_path = '/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/versions/1/datasets'
train_image_dir = os.path.join(dataset_path, 'train_images')
train_mask_dir = os.path.join(dataset_path, 'train_masks')

# Create the dataset instance
train_dataset = CustomDataset(
    image_dir=train_image_dir,
    mask_dir=train_mask_dir,
    transform=transform
)

# Check dataset length and sample
print(f"Number of training samples: {len(train_dataset)}")

# Test loading a sample
image, mask = train_dataset[0]
print(f"Image shape: {image.shape}, Mask shape: {mask.shape}")


Number of training samples: 8069
Image shape: torch.Size([3, 224, 224]), Mask shape: torch.Size([1, 224, 224])


In [ ]:
train_dataset = CustomDataset(
    image_dir=f"{dataset_path}/train_images",  # Adjust path
    mask_dir=f"{dataset_path}/train_masks",  # Adjust path
    transform=transform
)


In [ ]:
import os

train_images_path = f"{dataset_path}/train_images"
train_masks_path = f"{dataset_path}/train_masks"

print("Train Images Path Exists:", os.path.exists(train_images_path))
print("Train Masks Path Exists:", os.path.exists(train_masks_path))


Train Images Path Exists: True
Train Masks Path Exists: True


In [ ]:
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch

class CustomDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(image_dir))  # Assumes images and masks are sorted in the same order
        self.mask_files = sorted(os.listdir(mask_dir))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load the image
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(image_path).convert('RGB')  # Convert to RGB if the images are grayscale

        # Load the mask
        mask_path = os.path.join(self.mask_dir, self.mask_files[idx])
        mask = Image.open(mask_path).convert('L')  # Convert to grayscale for the mask

        if self.transform:
            # Apply transformations to the image
            image = self.transform(image)

            # Resize and convert mask to tensor
            mask = transforms.Resize((224, 224))(mask)
            mask = transforms.ToTensor()(mask)  # Convert mask to tensor without normalizing

        return image, mask

# Set the paths
dataset_path = '/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/versions/1/datasets'
train_image_dir = os.path.join(dataset_path, 'train_images')
train_mask_dir = os.path.join(dataset_path, 'train_masks')

# Define the transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

# Load the dataset using the custom dataset class
train_dataset = CustomDataset(image_dir=train_image_dir, mask_dir=train_mask_dir, transform=transform)

# Create the DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Test if everything is working
for images, masks in train_loader:
    print(images.shape, masks.shape)  # Should print the shapes of the images and masks
    break  # Just print one batch


torch.Size([32, 3, 224, 224]) torch.Size([32, 1, 224, 224])


In [ ]:
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch

class CustomDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(image_dir))  # Assumes images and masks are sorted in the same order
        self.mask_files = sorted(os.listdir(mask_dir))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load the image
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(image_path).convert('RGB')  # Convert to RGB if the images are grayscale

        # Load the mask
        mask_path = os.path.join(self.mask_dir, self.mask_files[idx])
        mask = Image.open(mask_path).convert('L')  # Convert to grayscale for the mask

        if self.transform:
            # Apply transformations to both image and mask
            image = self.transform(image)
            mask = transforms.Resize((224, 224))(mask)  # Resize mask to 224x224
            mask = transforms.ToTensor()(mask)  # Convert mask to tensor

        return image, mask

# Set the paths
dataset_path = '/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets'
train_image_dir = os.path.join(dataset_path, 'train_images')
train_mask_dir = os.path.join(dataset_path, 'train_masks')

# Define the transformations for images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset using the custom dataset class
train_dataset = CustomDataset(image_dir=train_image_dir, mask_dir=train_mask_dir, transform=transform)

# Create the DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Test if everything is working
for images, masks in train_loader:
    print(images.shape, masks.shape)  # Should print the shapes of the images and masks
    break  # Just print one batch


torch.Size([32, 3, 224, 224]) torch.Size([32, 1, 224, 224])


In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm

class NKUTModel(nn.Module):
    def __init__(self, num_classes):
        super(NKUTModel, self).__init__()
        self.base_model = models.resnet50(pretrained=True)  # Using ResNet-50 as the base model
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, num_classes)  # Modify output layer

    def forward(self, x):
        return self.base_model(x)

# Device setup (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch

class CustomDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(image_dir))  # Assumes images and masks are sorted in the same order
        self.mask_files = sorted(os.listdir(mask_dir))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load the image
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(image_path).convert('RGB')  # Convert to RGB if the images are grayscale

        # Load the mask
        mask_path = os.path.join(self.mask_dir, self.mask_files[idx])
        mask = Image.open(mask_path).convert('L')  # Convert to grayscale for the mask

        if self.transform:
            # Apply transformations to both image and mask
            image = self.transform(image)
            mask = transforms.Resize((224, 224))(mask)  # Resize mask to 224x224
            mask = transforms.ToTensor()(mask)  # Convert mask to tensor

        return image, mask

# Set the paths
dataset_path = '/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets'
train_image_dir = os.path.join(dataset_path, 'train_images')
train_mask_dir = os.path.join(dataset_path, 'train_masks')

# Define the transformations for images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset using the custom dataset class
train_dataset = CustomDataset(image_dir=train_image_dir, mask_dir=train_mask_dir, transform=transform)

# Create the DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Test if everything is working
for images, masks in train_loader:
    print(images.shape, masks.shape)  # Should print the shapes of the images and masks
    break  # Just print one batch


torch.Size([32, 3, 224, 224]) torch.Size([32, 1, 224, 224])


 Model Setup (ResNet50 for Segmentation)


In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class NKUTModel(nn.Module):
    def __init__(self, num_classes=1):  # Default for binary segmentation, can change
        super(NKUTModel, self).__init__()
        self.base_model = models.resnet50(pretrained=True)  # Using ResNet-50 as the base model
        self.base_model.fc = nn.Identity()  # Remove the fully connected layer

        # Add custom layers for segmentation
        self.conv1 = nn.Conv2d(2048, 1024, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(1024, num_classes, kernel_size=1)  # Output layer for segmentation
        self.upsample = nn.Upsample(scale_factor=32, mode='bilinear', align_corners=True)  # Upsample to original size

    def forward(self, x):
        x = self.base_model(x)  # Extract features using ResNet50
        x = self.relu(self.conv1(x))  # Apply convolution and ReLU
        x = self.conv2(x)  # Segmentation layer
        x = self.upsample(x)  # Upsample the result
        return x

# Initialize the model
model = NKUTModel(num_classes=1).to(device)  # You can change num_classes if you have multi-class segmentation



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class NKUTModel(nn.Module):
    def __init__(self, num_classes=1):  # Default for binary segmentation, can change
        super(NKUTModel, self).__init__()

        # Use the new weights argument for ResNet-50
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.base_model.fc = nn.Identity()  # Remove the fully connected layer

        # Add custom layers for segmentation
        self.conv1 = nn.Conv2d(2048, 1024, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(1024, num_classes, kernel_size=1)  # Output layer for segmentation
        self.upsample = nn.Upsample(scale_factor=32, mode='bilinear', align_corners=True)  # Upsample to original size

    def forward(self, x):
        x = self.base_model(x)  # Extract features using ResNet50
        x = self.relu(self.conv1(x))  # Apply convolution and ReLU
        x = self.conv2(x)  # Segmentation layer
        x = self.upsample(x)  # Upsample the result
        return x

# Initialize the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NKUTModel(num_classes=1).to(device)  # You can change num_classes if you have multi-class segmentation


In [ ]:
from torchvision import models

class NKUTModel(nn.Module):
    def __init__(self, num_classes=1):  # Default for binary segmentation, can change
        super(NKUTModel, self).__init__()

        # Use the updated way to load pretrained weights
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)  # Updated weights argument
        self.base_model.fc = nn.Identity()  # Remove the fully connected layer

        # Add custom layers for segmentation
        self.conv1 = nn.Conv2d(2048, 1024, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(1024, num_classes, kernel_size=1)  # Output layer for segmentation
        self.upsample = nn.Upsample(scale_factor=32, mode='bilinear', align_corners=True)  # Upsample to original size

    def forward(self, x):
        x = self.base_model(x)  # Extract features using ResNet50
        x = self.relu(self.conv1(x))  # Apply convolution and ReLU
        x = self.conv2(x)  # Segmentation layer
        x = self.upsample(x)  # Upsample the result
        return x

# Initialize the model
model = NKUTModel(num_classes=1).to(device)  # You can change num_classes if you have multi-class segmentation


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 153MB/s]


In [ ]:
# For binary segmentation (if masks have values 0 or 1)
criterion = nn.BCEWithLogitsLoss()

# For multi-class segmentation (if your masks have class values > 1)
# criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
epochs = 10  # Number of training epochs
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Iterate over the training data
    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Calculate loss
        loss = criterion(outputs, masks)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [32, 2048]

he error you encountered suggests that the input to the convolutional layer is not in the expected shape. The model expects a 4D tensor with the shape [batch_size, channels, height, width], but it received a 2D tensor with the shape [32, 2048], which is likely the result of flattening the output from the fully connected layer.

This issue likely arises because, after modifying the ResNet50 model, you're trying to use it for segmentation, but the model is designed for classification, where the output is a flat vector instead of a feature map suitable for segmentation.

Solution
Since you're doing segmentation, you should modify the network in a way that produces a spatial output (a 4D tensor) instead of flattening the output into a vector. You can use a fully convolutional network (FCN) style output.

How to Modify the Model for Segmentation
Instead of directly replacing the final fully connected layer (self.base_model.fc), you should replace it with a layer that outputs a segmentation mask of the same spatial size as the input image (e.g., 224x224 for your case).

Here’s how you can modify the NKUTModel to output a segmentation mask:

Replace the Fully Connected Layer with a Transposed Convolution Layer:

This will help in upscaling the output back to the image dimensions.
Add an Upsampling Layer:

Use a ConvTranspose2d layer to upsample the output to the desired resolution.

In [ ]:
!pip install torch torchvision


In [ ]:
import torch
import torch.nn as nn
from torchvision import models

class NKUTModel(nn.Module):
    def __init__(self, num_classes=1):  # num_classes = 1 for binary segmentation
        super(NKUTModel, self).__init__()

        # Load the pre-trained ResNet50 model
        self.base_model = models.resnet50(weights="IMAGENET1K_V1")  # Use ImageNet weights

        # Remove the fully connected layer (fc) to preserve the feature maps
        self.base_model.fc = nn.Identity()  # No need for classification layer

        # Add a transposed convolution layer to upsample the feature maps to the original image size
        self.conv_transpose = nn.ConvTranspose2d(
            in_channels=2048,        # The output channels of ResNet50 (2048)
            out_channels=num_classes,  # Number of output classes for segmentation
            kernel_size=16,          # Size of the convolution kernel
            stride=8,                # Stride of the convolution
            padding=4                # Padding to maintain the size after convolution
        )

    def forward(self, x):
        # Extract features from ResNet50
        features = self.base_model(x)  # Output size is (batch_size, 2048)

        # We need to reshape the feature map to have spatial dimensions (H, W)
        features = features.unsqueeze(-1).unsqueeze(-1)  # Add spatial dimensions: (batch_size, 2048, 1, 1)

        # Perform the upsampling (transposed convolution)
        segmentation_mask = self.conv_transpose(features)  # Output shape: (batch_size, num_classes, H, W)

        return segmentation_mask


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model
model = NKUTModel(num_classes=1)  # Binary segmentation output (you can change num_classes if needed)
model = model.to(device)

# Print the model architecture
print(model)


NKUTModel(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
     

In [ ]:
cd /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets


/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets


In [ ]:
ls


_classes.csv  test_images/  test_masks/  train_images/  train_masks/  val_images/  val_masks/


In [ ]:
train_image_dir = '/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets/train_images'
train_mask_dir = '/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets/train_masks'

train_dataset = CustomDataset(image_dir=train_image_dir, mask_dir=train_mask_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [ ]:
# List the contents of train_images directory
!ls /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets/train_images

# List the contents of train_masks directory
!ls /root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets/train_masks


cases1004_100.png  cases145_442.png   cases1860_64.png	 cases2246_65.png   cases2657_114.png
cases1004_104.png  cases145_444.png   cases1860_65.png	 cases2246_66.png   cases2657_115.png
cases1004_105.png  cases145_445.png   cases1860_66.png	 cases2246_67.png   cases2657_116.png
cases1004_106.png  cases145_446.png   cases1860_69.png	 cases2246_70.png   cases2657_117.png
cases1004_107.png  cases145_448.png   cases1860_70.png	 cases2246_72.png   cases2657_118.png
cases1004_108.png  cases145_450.png   cases1870_37.png	 cases2246_74.png   cases2657_119.png
cases1004_109.png  cases145_451.png   cases1870_38.png	 cases2246_75.png   cases2657_120.png
cases1004_110.png  cases145_453.png   cases1870_39.png	 cases2246_76.png   cases2657_122.png
cases1004_111.png  cases145_455.png   cases1870_42.png	 cases2246_77.png   cases2657_124.png
cases1004_112.png  cases145_457.png   cases1870_43.png	 cases2246_78.png   cases2657_126.png
cases1004_113.png  cases145_458.png   cases1870_44.png	 cases2246_79.p

In [ ]:
from torch.utils.data import DataLoader
from torchvision import transforms
import os
from PIL import Image
import torch
from torch.utils.data import Dataset

# Custom Dataset class (ensure this is correctly defined as before)
class CustomDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(image_dir))  # Assumes images and masks are sorted in the same order
        self.mask_files = sorted(os.listdir(mask_dir))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(image_path).convert('RGB')  # Convert to RGB if grayscale
        mask_path = os.path.join(self.mask_dir, self.mask_files[idx])
        mask = Image.open(mask_path).convert('L')  # Convert to grayscale for the mask

        if self.transform:
            image = self.transform(image)
            mask = transforms.Resize((224, 224))(mask)  # Resize mask to 224x224
            mask = transforms.ToTensor()(mask)  # Convert mask to tensor

        return image, mask

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Corrected directories for your image and mask datasets
train_image_dir = '/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets/train_images'
train_mask_dir = '/root/.cache/kagglehub/datasets/juliusshrestha/wilms-tumorct/datasets/train_masks'

# Create the dataset and DataLoader
train_dataset = CustomDataset(image_dir=train_image_dir, mask_dir=train_mask_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.middle = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, out_channels, kernel_size=3, padding=1)
        )

    def forward(self, x):
        x1 = self.encoder(x)
        x2 = self.middle(x1)
        x3 = self.decoder(x2)
        return x3


In [ ]:
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.middle = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, out_channels, kernel_size=3, padding=1)
        )

    def forward(self, x):
        x1 = self.encoder(x)
        x2 = self.middle(x1)
        x3 = self.decoder(x2)

        # Resize the output to match the mask size (224x224)
        x3_resized = F.interpolate(x3, size=(224, 224), mode='bilinear', align_corners=False)

        return x3_resized


In [ ]:
import torch.nn.functional as F

def evaluate_model(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    all_predictions = []
    all_ground_truth = []

    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)

            # Get model predictions
            outputs = model(images)  # Outputs may have dimensions [batch_size, 1, 112, 112]

            # Resize outputs to match ground truth mask dimensions (224x224)
            outputs_resized = F.interpolate(outputs, size=(224, 224), mode='bilinear', align_corners=False)

            # Append predictions and ground truth for metric calculations
            all_predictions.append(outputs_resized.cpu())
            all_ground_truth.append(masks.cpu())

    # Concatenate predictions and ground truth
    all_predictions = torch.cat(all_predictions, dim=0)
    all_ground_truth = torch.cat(all_ground_truth, dim=0)

    # Binarize predictions and ground truth (threshold = 0.5)
    predictions_binary = (all_predictions > 0.5).float()
    ground_truth_binary = (all_ground_truth > 0.5).float()

    # Calculate Dice Coefficient
    intersection = (predictions_binary * ground_truth_binary).sum()
    union = predictions_binary.sum() + ground_truth_binary.sum()
    dice_score = (2 * intersection) / (union + 1e-6)  # Add epsilon to avoid division by zero

    return dice_score.item()


In [ ]:
# Evaluate the model on the training or validation set
dice_score = evaluate_model(model, train_loader, device)
print(f'Dice Score: {dice_score:.4f}')


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [32, 2048]

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class NKUTModel(nn.Module):
    def __init__(self, num_classes=1):  # Default for binary segmentation, can change
        super(NKUTModel, self).__init__()

        # Use ResNet-50 as the backbone
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.base_layers = list(self.base_model.children())[:-2]  # Extract layers until the final convolutional layer
        self.backbone = nn.Sequential(*self.base_layers)

        # Custom segmentation layers
        self.conv1 = nn.Conv2d(2048, 1024, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(1024, num_classes, kernel_size=1)  # Output layer for segmentation
        self.upsample = nn.Upsample(scale_factor=32, mode='bilinear', align_corners=True)  # Upsample to match input size

    def forward(self, x):
        x = self.backbone(x)  # Extract features using ResNet-50 backbone
        x = self.relu(self.conv1(x))  # Apply convolution and ReLU
        x = self.conv2(x)  # Segmentation layer
        x = self.upsample(x)  # Upsample the result
        return x


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class NKUTModel(nn.Module):
    def __init__(self, num_classes=1):  # Default for binary segmentation
        super(NKUTModel, self).__init__()
        # Load the ResNet-50 backbone
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.base_layers = list(self.base_model.children())[:-2]  # Remove FC and AvgPool layers

        # Feature extractor with preserved spatial dimensions
        self.feature_extractor = nn.Sequential(*self.base_layers)

        # Add custom segmentation layers
        self.conv1 = nn.Conv2d(2048, 1024, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(1024, num_classes, kernel_size=1)  # Output layer for segmentation

        # Upsample to original image size
        self.upsample = nn.Upsample(scale_factor=32, mode='bilinear', align_corners=True)

    def forward(self, x):
        # Extract features from the backbone
        x = self.feature_extractor(x)  # Output shape: [batch_size, 2048, H/32, W/32]

        # Apply custom segmentation layers
        x = self.relu(self.conv1(x))  # Shape: [batch_size, 1024, H/32, W/32]
        x = self.conv2(x)             # Shape: [batch_size, num_classes, H/32, W/32]

        # Upsample to match the input image size
        x = self.upsample(x)          # Shape: [batch_size, num_classes, H, W]
        return x

# Initialize the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NKUTModel(num_classes=1).to(device)


In [ ]:
def evaluate_model_with_metrics(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    all_predictions = []
    all_ground_truth = []

    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)

            # Get model predictions
            outputs = model(images)  # Outputs should have dimensions [batch_size, 1, 224, 224]

            # Binarize predictions and ground truth (threshold = 0.5)
            predictions_binary = (outputs > 0.5).float()
            ground_truth_binary = (masks > 0.5).float()

            # Append predictions and ground truth for metric calculations
            all_predictions.append(predictions_binary.cpu())
            all_ground_truth.append(ground_truth_binary.cpu())

    # Concatenate predictions and ground truth
    all_predictions = torch.cat(all_predictions, dim=0)
    all_ground_truth = torch.cat(all_ground_truth, dim=0)

    # Calculate metrics
    TP = (all_predictions * all_ground_truth).sum()  # True Positives
    FP = ((all_predictions == 1) & (all_ground_truth == 0)).sum()  # False Positives
    FN = ((all_predictions == 0) & (all_ground_truth == 1)).sum()  # False Negatives
    TN = ((all_predictions == 0) & (all_ground_truth == 0)).sum()  # True Negatives

    # Sensitivity (Recall)
    recall = TP / (TP + FN + 1e-6)  # Add epsilon to avoid division by zero

    # Precision
    precision = TP / (TP + FP + 1e-6)

    # Accuracy
    accuracy = (TP + TN) / (TP + TN + FP + FN + 1e-6)

    # Dice Coefficient
    intersection = TP
    union = TP + FP + FN
    dice_score = (2 * intersection) / (union + 1e-6)  # Add epsilon to avoid division by zero

    return {
        "Dice": dice_score.item(),
        "Recall (Sensitivity)": recall.item(),
        "Precision": precision.item(),
        "Accuracy": accuracy.item(),
    }

# Evaluate the model
metrics = evaluate_model_with_metrics(model, train_loader, device)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")
